![CoSAppLogo](../images/cosapp.svg)

<font color='orange'>**CoSApp**</font> tutorials on multimode systems

# Bouncing Ball

In [ ]:
from cosapp.base import System
import numpy as np


class PointMassDynamics(System):
    """Free fall of a point mass, with friction"""
    def setup(self):
        self.add_inward('mass', 1.2, desc='Mass')
        self.add_inward('cf', 0.1, desc='Friction coefficient')
        self.add_inward('g', np.r_[0, 0, -9.81], desc='Uniform acceleration field')

        self.add_outward('a', np.zeros(3))
        self.add_transient('v', der='a')
        self.add_transient('x', der='v')

    def compute(self):
        self.a = self.g - (self.cf / self.mass * np.linalg.norm(self.v)) * self.v


class PointMassGround(System):
    """Bouncing point mass"""
    def setup(self):
        self.add_child(PointMassDynamics('dyn'), pulling=['x', 'v', 'a', 'mass', 'cf', 'g'])
        self.add_event('rebound', trigger="x[2] <= 0")
        self.add_inward('cr', 0.0, desc="Rebound loss coefficient", limits=(0, 1))

    def transition(self):
        cr = max(0, min(self.cr, 1))
        if self.rebound.present:
            v = self.v
            if abs(v[2]) < 1e-6:
                v[2] = 0
            else:
                v[2] *= -(1 - cr)


In [ ]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder

point = PointMassGround('point')
driver = point.add_driver(RungeKutta(order=3, dt=0.01))
driver.time_interval = (0, 10)

# Add a recorder to capture time evolution in a dataframe
driver.add_recorder(DataFrameRecorder(includes=['x', 'v', 'a']), period=0.05)

# Initial conditions
x0 = [0, 0, 0]
v0 = [8, 0, 9.5]

# Define a simulation scenario
driver.set_scenario(
    init = {'x': np.array(x0), 'v': np.array(v0)},
    values = {
        'mass': 1.5,
        'cf': 0.2,
        'cr': 0.02,
    },
)

point.run_drivers()

# Retrieve recorded data
data = driver.recorder.export_data()
data = data.drop(['Section', 'Status', 'Error code'], axis=1)
time = np.asarray(data['time'])
traj = np.asarray(data['x'].tolist())


In [ ]:
# Plot results
import plotly.graph_objs as go

traces = [
    go.Scatter(
        x = traj[:, 0],
        y = traj[:, 2],
        mode = 'lines',
        name = 'numerical',
        line = dict(color='red'),
    )
]
layout = go.Layout(
    title = "Trajectory",
    xaxis = dict(title="x"),
    yaxis = dict(
        title = "z",
        scaleanchor = "x",
        scaleratio = 1,
    ),
    hovermode = "x",
)
go.Figure(data=traces, layout=layout)

In the next cell, we filter the recorded data in the vicinity of a rebound.
Notice that there are two entries at event time $t \approx 1.457$, showing the discontinuity in velocity (all other quantities are here continuous).

In [ ]:
data[data['time'].between(1.4, 1.6)]

Time driver property `recorded_events` contains a list of all recorded events.
Each element of the list is a named tuple `EventRecord(time, events)`, containing the occurrence time, and the list of cascading events at that time, starting by the primitive event.

In [ ]:
driver.recorded_events

Time driver property `event_data` contains the same fields as the recorder, but only at event times:

In [ ]:
driver.event_data.drop(['Section', 'Status', 'Error code'], axis=1)